In [2]:
#!pip install iterative-stratification

In [1]:
import pandas as pd
import numpy as np

import util

In [2]:
train_df = util.read_trainset()

In [3]:
X = train_df.index.values
y = train_df['Label'].values[:, 1:] # remove `any` column

In [4]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import numpy as np

#X = np.array([[1,2], [3,4], [1,2], [3,4], [1,2], [3,4], [1,2], [3,4]])
#y = np.array([[0,0], [0,0], [0,1], [0,1], [1,1], [1,1], [1,0], [1,0]])

kf = MultilabelStratifiedKFold(n_splits=5, random_state=0)

folds = np.zeros(shape=(len(y),), dtype=np.uint8)

for fold_no, (train_index, test_index) in enumerate(kf.split(X, y)):
    #print(f'fold: {fold_no} TRAIN: {train_index} TEST: {test_index}')
    #X_train, X_test = X[train_index], X[test_index]
    #y_train, y_test = y[train_index], y[test_index]
    folds[test_index] = fold_no
    

In [5]:
print(len(folds))
for fold_no in range(5):
    print(np.sum(folds==fold_no)/len(folds))

674257
0.20000088986840328
0.1999994067543978
0.1999994067543978
0.20000088986840328
0.1999994067543978


In [6]:
from collections import Counter
label_sets = Counter()
for label in y:
    x = tuple(label)
    label_sets[x] += 1
print(len(label_sets))    

32


In [7]:
label_sets.most_common(10)

[((0, 0, 0, 0, 0), 577154),
 ((0, 0, 0, 0, 1), 28909),
 ((0, 0, 0, 1, 0), 14717),
 ((0, 1, 0, 0, 0), 13932),
 ((0, 0, 1, 0, 0), 8914),
 ((0, 1, 1, 0, 0), 6330),
 ((0, 0, 0, 1, 1), 4654),
 ((0, 1, 0, 1, 0), 3611),
 ((0, 0, 1, 1, 0), 3226),
 ((0, 1, 0, 0, 1), 2952)]

In [9]:
# counts of most common label sets accross folds
for fold_no in range(5):
    y_fold = y[folds==fold_no]
    for label_set, _ in label_sets.most_common(10):        
        amount = np.sum([1 for x in y_fold if tuple(x) == label_set])
        print(f'fold {fold_no} label {label_set} amount: {amount}')

fold 0 label (0, 0, 0, 0, 0) amount: 115357
fold 0 label (0, 0, 0, 0, 1) amount: 5832
fold 0 label (0, 0, 0, 1, 0) amount: 2990
fold 0 label (0, 1, 0, 0, 0) amount: 2822
fold 0 label (0, 0, 1, 0, 0) amount: 1817
fold 0 label (0, 1, 1, 0, 0) amount: 1232
fold 0 label (0, 0, 0, 1, 1) amount: 906
fold 0 label (0, 1, 0, 1, 0) amount: 698
fold 0 label (0, 0, 1, 1, 0) amount: 641
fold 0 label (0, 1, 0, 0, 1) amount: 588
fold 1 label (0, 0, 0, 0, 0) amount: 115495
fold 1 label (0, 0, 0, 0, 1) amount: 5758
fold 1 label (0, 0, 0, 1, 0) amount: 2901
fold 1 label (0, 1, 0, 0, 0) amount: 2753
fold 1 label (0, 0, 1, 0, 0) amount: 1767
fold 1 label (0, 1, 1, 0, 0) amount: 1268
fold 1 label (0, 0, 0, 1, 1) amount: 937
fold 1 label (0, 1, 0, 1, 0) amount: 739
fold 1 label (0, 0, 1, 1, 0) amount: 638
fold 1 label (0, 1, 0, 0, 1) amount: 588
fold 2 label (0, 0, 0, 0, 0) amount: 115434
fold 2 label (0, 0, 0, 0, 1) amount: 5722
fold 2 label (0, 0, 0, 1, 0) amount: 2901
fold 2 label (0, 1, 0, 0, 0) amount:

In [11]:
folds_df = pd.DataFrame()
folds_df['image'] = X
folds_df['fold'] = folds
for c in train_df['Label'].columns:
    folds_df[c] = train_df['Label'][c].values
folds_df.to_csv('../data/stage_1_train_folds.csv', index=False)


In [12]:
!head ../data/stage_1_train_folds.csv

image,fold,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
ID_000039fa0,1,0,0,0,0,0,0
ID_00005679d,1,0,0,0,0,0,0
ID_00008ce3c,1,0,0,0,0,0,0
ID_0000950d7,1,0,0,0,0,0,0
ID_0000aee4b,4,0,0,0,0,0,0
ID_0000f1657,1,0,0,0,0,0,0
ID_000178e76,4,0,0,0,0,0,0
ID_00019828f,1,0,0,0,0,0,0
ID_0001dcc25,4,0,0,0,0,0,0
